In [1]:
! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  4  955k    4 44735    0     0  33498      0  0:00:29  0:00:01  0:00:28 33534
 33  955k   33  317k    0     0   141k      0  0:00:06  0:00:02  0:00:04  141k
 70  955k   70  671k    0     0   196k      0  0:00:04  0:00:03  0:00:01  196k
100  955k  100  955k    0     0   230k      0  0:00:04  0:00:04 --:--:--  230k


In [2]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [3]:
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [4]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [5]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")
     

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [9]:
pip install torch

   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 1.0/204.1 MB 6.3 MB/s eta 0:00:33
    --------------------------------------- 2.6/204.1 MB 8.9 MB/s eta 0:00:23
    --------------------------------------- 4.7/204.1 MB 7.9 MB/s eta 0:00:26
   - -------------------------------------- 5.2/204.1 MB 7.2 MB/s eta 0:00:28
   - -------------------------------------- 6.8/204.1 MB 7.1 MB/s eta 0:00:28
   - -------------------------------------- 7.6/204.1 MB 6.3 MB/s eta 0:00:31
   - -------------------------------------- 9.2/204.1 MB 6.6 MB/s eta 0:00:30
   - -------------------------------------- 10.0/204.1 MB 6.4 MB/s eta 0:00:31
   -- ------------------------------------- 10.7/204.1 MB 6.2 MB/s eta 0:00:32
   -- ------------------------------------- 11.3/204.1 MB 5.7 MB/s eta 0:00:34
   -- ------------------------------------- 12.6/204.1 MB 5.6 MB/s eta 0:00:35
   -- ------------------------------------- 13.6/204.1 MB 5.6 MB/s 

In [14]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.item_factors = torch.nn.Embedding(n_items, n_factors)
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

In [15]:
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [28]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
if cuda:
    model = model.cuda()

loss_fn = torch.nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: False
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0261, 0.0215, 0.0395,  ..., 0.0425, 0.0124, 0.0216],
        [0.0310, 0.0238, 0.0010,  ..., 0.0484, 0.0007, 0.0396],
        [0.0455, 0.0343, 0.0400,  ..., 0.0462, 0.0176, 0.0262],
        ...,
        [0.0393, 0.0033, 0.0487,  ..., 0.0277, 0.0461, 0.0113],
        [0.0024, 0.0147, 0.0171,  ..., 0.0330, 0.0391, 0.0184],
        [0.0325, 0.0205, 0.0400,  ..., 0.0262, 0.0219, 0.0025]])
item_factors.weight tensor([[0.0307, 0.0135, 0.0265,  ..., 0.0102, 0.0285, 0.0056],
        [0.0031, 0.0235, 0.0093,  ..., 0.0355, 0.0219, 0.0081],
        [0.0004, 0.0143, 0.0443,  ..., 0.0426, 0.0062, 0.0291],
        ...,
        [0.0027, 0.0007, 0.0184,  ..., 0.0176, 0.0313, 0.0362],
        [0.0250, 0.0060, 0.0210,  ..., 0.0366, 0.0297, 0.0080],
        [0.0435, 0.0094, 0.0278,  ..., 0.0299, 0.0369, 0.0289]])


In [31]:
from tqdm import tqdm 

for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
        if cuda:
            x, y = x.cuda(), y.cuda()
        optimizer.zero_grad()
        outputs = model(x)
        loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))






                                                                                                                                                                            | 0/128 [00:00<?, ?it/s]



                                                                                                                                                                    | 1/128 [00:01<02:18,  1.09s/it]

iter #0 Loss: 0.32721169357221136






                                                                                                                                                                    | 2/128 [00:02<02:17,  1.09s/it]

iter #1 Loss: 0.32670211531032767






                                                                                                                                                                    | 3/128 [00:03<02:15,  1.09s/it]

...

iter #125 Loss: 0.3009238020342014






████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 127/128 [02:16<00:01,  1.08s/it]

iter #126 Loss: 0.3008948089696732






100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 128/128 [02:17<00:00,  1.08s/it]

iter #127 Loss: 0.30086155795506414


In [32]:
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data

user_factors.weight tensor([[ 1.1267,  1.5610,  1.5968,  ...,  0.4851,  1.0235,  0.7394],
        [ 1.1193,  2.0490,  0.0356,  ...,  1.0188,  0.4372,  1.1902],
        [ 0.7367,  1.8155,  1.3284,  ...,  2.0869, -2.3494,  0.3161],
        ...,
        [ 1.6868,  0.2823,  0.5313,  ...,  1.6125,  1.2129, -0.5139],
        [ 1.5355,  0.7007,  1.2334,  ...,  0.2577,  0.6127,  1.1014],
        [ 1.0896,  1.4784,  1.4460,  ...,  1.0787,  0.6065,  0.8787]])
item_factors.weight tensor([[ 0.2224,  0.7647,  0.6119,  ...,  0.3477,  0.7031,  0.4358],
        [ 0.6432,  0.2322,  0.1682,  ..., -0.0184,  0.3461,  0.5648],
        [ 0.5105,  0.6172,  0.7974,  ...,  0.3648,  0.2195,  0.3756],
        ...,
        [ 0.3694,  0.3652,  0.3834,  ...,  0.3830,  0.4001,  0.4039],
        [ 0.4416,  0.4219,  0.4402,  ...,  0.4525,  0.4428,  0.4216],
        [ 0.4573,  0.4241,  0.4419,  ...,  0.4424,  0.4451,  0.4404]])


In [33]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [34]:
len(trained_movie_embeddings)

9724

In [35]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [36]:
for cluster in range(10):
    print("Cluster #{}".format(cluster))
    movs = []
    for movidx in np.where(kmeans.labels_ == cluster)[0]:
        movid = train_set.idx2movieid[movidx]
        rat_count = len(ratings_df.loc[ratings_df['movieId'] == movid])
        movs.append((movie_names[movid], rat_count))
    for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
        print("\t", mov[0])

Cluster #0
	 Twister (1996)
	 Home Alone (1990)
	 Fifth Element, The (1997)
	 Harry Potter and the Chamber of Secrets (2002)
	 Harry Potter and the Prisoner of Azkaban (2004)
	 Star Trek: First Contact (1996)
	 Back to the Future Part II (1989)
	 Broken Arrow (1996)
	 Nutty Professor, The (1996)
	 Mr. Holland's Opus (1995)
Cluster #1
	 Batman Forever (1995)
	 Mission: Impossible II (2000)
	 Honey, I Shrunk the Kids (1989)
	 Hot Shots! Part Deux (1993)
	 Space Jam (1996)
	 Johnny Mnemonic (1995)
	 Arachnophobia (1990)
	 Father of the Bride Part II (1995)
	 Nine Months (1995)
	 Mortal Kombat (1995)
Cluster #2
	 Star Wars: Episode I - The Phantom Menace (1999)
	 Four Weddings and a Funeral (1994)
	 Star Wars: Episode II - Attack of the Clones (2002)
	 Matrix Revolutions, The (2003)
	 Scream (1996)
	 Dead Man Walking (1995)
	 Sense and Sensibility (1995)
	 Mask of Zorro, The (1998)
	 A.I. Artificial Intelligence (2001)
	 Moulin Rouge (2001)
Cluster #3
	 Terminator 2: Judgment Day (1991)
	 